In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"   #(xxxx is your specific GPU ID)

In [2]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Activation, Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from MyEarlyStopping import MyEarlyStopping
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# from keras.optimizers import adam

import timeit
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from sklearn.preprocessing import LabelEncoder  

# load dataset 

In [3]:
train_number = 5
train_image = 20 #10:20%, 20: 40%, 30:60%
train_image_s2 = 30 #10:20%, 20: 40%, 30:60%

## Training dataset (40%)

In [4]:
train_data = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_data.flow_from_directory('/data/home/cou/yuli5/Audio/Comparision/Different_Inputs/Audio_MFCC/train',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                               shuffle=False)

Found 1714 images belonging to 10 classes.


In [5]:
image_names = train_generator.filenames
image_no = [i.split("/")[1].split("_")[2].split(".")[0] for i in image_names]
image_no = np.array(list(map(int, image_no)))
ALL_participant_class = [i.split("/")[1].split("_")[1] for i in image_names]
ALL_participant_class = np.array(list(map(int, ALL_participant_class)))
command_class = train_generator.classes
All_participant_class = tf.keras.utils.to_categorical(ALL_participant_class-1, num_classes=train_number)
All_command_class = tf.keras.utils.to_categorical(command_class, num_classes=10)
All_command_uniform = All_command_class*0+1/10

In [6]:
All_Inputs = [next(train_generator)[0][0] for _ in range(len(train_generator))]
All_Inputs = np.array(All_Inputs)

In [7]:
select_indexs_train = ((image_no<train_image)&(ALL_participant_class!=2))|((image_no<train_image_s2)&(ALL_participant_class==2))
Train_Inputs = All_Inputs[select_indexs_train]
Train_participant_class = All_participant_class[select_indexs_train]
Train_participant_uniform = Train_participant_class*0+1/train_number
Train_command_class = All_command_class[select_indexs_train]
Train_command_uniform = Train_command_class*0+1/10
# sum(Train_participant_class)


## Validation dataset (20%)

In [8]:
val_data = ImageDataGenerator(preprocessing_function=preprocess_input)
val_generator = val_data.flow_from_directory('/data/home/cou/yuli5/Audio/Comparision/Different_Inputs/Audio_MFCC/validation',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                               shuffle=False)

Found 543 images belonging to 10 classes.


In [9]:
image_names = val_generator.filenames
participant_class = [i.split("/", 1)[1].split("_")[1] for i in image_names]
participant_class = np.array(list(map(int, participant_class)))
command_class = val_generator.classes
Val_participant_class = tf.keras.utils.to_categorical(participant_class-1, num_classes=train_number)
Val_participant_uniform = Val_participant_class*0+1/train_number
Val_command_class = tf.keras.utils.to_categorical(command_class, num_classes=10)
Val_command_uniform = Val_command_class*0+1/10

In [10]:
Val_Inputs = [next(val_generator)[0][0] for _ in range(len(val_generator))]
Val_Inputs = np.array(Val_Inputs)

## Test dataset (20%)

In [11]:
test_data = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_data.flow_from_directory('/data/home/cou/yuli5/Audio/Comparision/Different_Inputs/Audio_MFCC/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 543 images belonging to 10 classes.


In [12]:
image_names = test_generator.filenames
participant_class = [i.split("/", 1)[1].split("_")[1] for i in image_names]
test_unit_participant_class = np.array(list(map(int, participant_class)))
test_unit_command_class = test_generator.classes
Test_participant_class = tf.keras.utils.to_categorical(test_unit_participant_class-1, num_classes=train_number)
Test_participant_uniform = Test_participant_class*0+1/train_number
Test_command_class = tf.keras.utils.to_categorical(test_unit_command_class, num_classes=10)
Test_command_uniform = Test_command_class*0+1/10

In [13]:
Test_Inputs = [next(test_generator)[0][0] for _ in range(len(test_generator))]
Test_Inputs = np.array(Test_Inputs)

In [14]:
Test_Inputs_1 = Test_Inputs[np.where(test_unit_participant_class == 1)]
Test_command_class_1 = Test_command_class[np.where(test_unit_participant_class == 1)]
Test_Inputs_2 = Test_Inputs[np.where(test_unit_participant_class == 2)]
Test_command_class_2 = Test_command_class[np.where(test_unit_participant_class == 2)]
Test_Inputs_3 = Test_Inputs[np.where(test_unit_participant_class == 3)]
Test_command_class_3 = Test_command_class[np.where(test_unit_participant_class == 3)]
Test_Inputs_4 = Test_Inputs[np.where(test_unit_participant_class == 4)]
Test_command_class_4 = Test_command_class[np.where(test_unit_participant_class == 4)]
Test_Inputs_5 = Test_Inputs[np.where(test_unit_participant_class == 5)]
Test_command_class_5 = Test_command_class[np.where(test_unit_participant_class == 5)]

# pd file store performance

In [15]:
# pd file store performance
Perfomance = pd.DataFrame(columns = ['Feature_Extract','Time','Partcp_Acc_p15','Command_Acc_p15','Partcp_Acc_p1','Command_Acc_p1',
                                      'Partcp_Acc_p2','Command_Acc_p2','Partcp_Acc_p3','Command_Acc_p3',
                                      'Partcp_Acc_p4','Command_Acc_p4','Partcp_Acc_p5','Command_Acc_p5'])

# Perfomance = pd.read_csv('Performance_0906_Inputs.csv')
# Perfomance
                                                   

# Create model 

In [16]:
actv_fun_1_1 = "relu" 
actv_fun_1_2 = "sigmoid"
shape_1_1 = 128
shape_1_2 = 256
actv_fun_2_1 = "sigmoid" 
actv_fun_2_2 = "sigmoid"
shape_2_1 = 512
shape_2_2 = 512


## Group model 

In [17]:
high_acc = 0
for run in range(0,10):
    # feature extraction layers
    vgg_model = VGG16(input_shape=(224, 224,3), include_top=False, weights="imagenet")
    for layer in vgg_model.layers:
        layer.trainable = False

    d = vgg_model.output.shape[-1] # dimension of last layer

    ###################### model 1 ###################### 
    layer_1_0 = tf.keras.layers.Dense(d,name="weight_1")(vgg_model.output) #times weight before flatten
    layer_1_1 = tf.keras.layers.Flatten(name='flatten_1')(layer_1_0)

    Dense_1_1 = tf.keras.layers.Dense(shape_1_1, activation=actv_fun_1_1,name='fc1_1')
    layer_1_2 = Dense_1_1(layer_1_1)
    Dense_1_2 = tf.keras.layers.Dense(shape_1_2, activation=actv_fun_1_2,name='fc1_2')
    layer_1_3 = Dense_1_2(layer_1_2)

    Dense_1_3 = tf.keras.layers.Dense(train_number, activation='softmax' ,name='participant_output')
    out_layer_1 = Dense_1_3(layer_1_3)

    ###################### model 2 ###################### 
    layer_2_0 = tf.keras.layers.Dense(d,name="weight_2")(vgg_model.output) #times weight before flatten
    layer_2_1 = tf.keras.layers.Flatten(name='flatten_2')(layer_2_0)

    Dense_2_1 = tf.keras.layers.Dense(shape_2_1, activation=actv_fun_2_1,name='fc2_1')
    layer_2_2  = Dense_2_1(layer_2_1)
    Dense_2_2 = tf.keras.layers.Dense(shape_2_2, activation=actv_fun_2_2,name='fc2_2')
    layer_2_3  = Dense_2_2(layer_2_2)

    Dense_2_3 = tf.keras.layers.Dense(10, activation='softmax',name='command_output')
    out_layer_2 = Dense_2_3(layer_2_3)

    ###################### model 1' ###################### 
    layer_1_2_  = Dense_2_1(layer_1_1)
    layer_1_3_  = Dense_2_2(layer_1_2_)
    out_layer_1_ = Dense_2_3(layer_1_3_)

    ###################### model 1' ###################### 
    layer_2_2_  = Dense_1_1(layer_2_1)
    layer_2_3_  = Dense_1_2(layer_2_2_)
    out_layer_2_ = Dense_1_3(layer_2_3_)

    vgg_model = tf.keras.Model(vgg_model.input, [out_layer_1,out_layer_2,out_layer_1_,out_layer_2_])


    # vgg_model.summary() 

    w_1, w_2, w_1_, w_2_ = 1,1,1,1
    ##################### training ############################
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    callbacks = [MyEarlyStopping(monitor1 = 'val_' + vgg_model.layers[-1].name+'_accuracy',
                                  monitor2 = 'val_' + vgg_model.layers[-2].name+'_accuracy',
                                  patience=5,restore_best_weights=True)]
    vgg_model.compile(optimizer=opt, loss=["categorical_crossentropy","categorical_crossentropy","mse","mse"],
                         loss_weights=[w_1, w_2, w_1_, w_2_], metrics=['accuracy'])


    start = timeit.default_timer()

    history1 = vgg_model.fit(Train_Inputs, 
                           {vgg_model.layers[-2].name:Train_participant_class, 
                            vgg_model.layers[-1].name:Train_command_class,
                            vgg_model.layers[-1].name+"_1":Train_command_uniform, 
                            vgg_model.layers[-2].name+"_1":Train_participant_uniform}, 
                            validation_data=(Val_Inputs,
                                             {vgg_model.layers[-2].name:Val_participant_class,
                                              vgg_model.layers[-1].name:Val_command_class,
                                              vgg_model.layers[-1].name+"_1":Val_command_uniform,
                                              vgg_model.layers[-2].name+"_1":Val_participant_uniform}), 
                           callbacks=callbacks,
                           batch_size=64,
                           epochs=10)

    for layer in vgg_model.layers[0:19]:
        layer.trainable = True

    history2 = vgg_model.fit(Train_Inputs, 
                               {vgg_model.layers[-2].name:Train_participant_class, 
                                vgg_model.layers[-1].name:Train_command_class,
                                vgg_model.layers[-1].name+"_1":Train_command_uniform, 
                                vgg_model.layers[-2].name+"_1":Train_participant_uniform}, 
                                validation_data=(Val_Inputs,
                                                 {vgg_model.layers[-2].name:Val_participant_class,
                                                  vgg_model.layers[-1].name:Val_command_class,
                                                  vgg_model.layers[-1].name+"_1":Val_command_uniform,
                                                  vgg_model.layers[-2].name+"_1":Val_participant_uniform}), 
                               callbacks=callbacks,
                               batch_size=64,
                               epochs=10)

    stop = timeit.default_timer()
    run_time = stop - start

    ##################### test performance ############################
    predictions = vgg_model.predict(Test_Inputs)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = test_unit_participant_class
    acc_p15_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)

    predictions = vgg_model.predict(Test_Inputs)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class
    acc_p15_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)


    # test on p1
    ## speaker classification
    predictions = vgg_model.predict(Test_Inputs_1)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([1]*len(predicted_classes))
    acc_p1_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = vgg_model.predict(Test_Inputs_1)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_1, axis=-1)
    acc_p1_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p2
    ## speaker classification
    predictions = vgg_model.predict(Test_Inputs_2)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([2]*len(predicted_classes))
    acc_p2_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = vgg_model.predict(Test_Inputs_2)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_2, axis=-1)
    acc_p2_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p3
    ## speaker classification
    predictions = vgg_model.predict(Test_Inputs_3)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([3]*len(predicted_classes))
    acc_p3_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = vgg_model.predict(Test_Inputs_3)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_3, axis=-1)
    acc_p3_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p4
    ## speaker classification
    predictions = vgg_model.predict(Test_Inputs_4)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([4]*len(predicted_classes))
    acc_p4_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = vgg_model.predict(Test_Inputs_4)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_4, axis=-1)
    acc_p4_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p5
    ## speaker classification
    predictions = vgg_model.predict(Test_Inputs_5)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([5]*len(predicted_classes))
    acc_p5_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = vgg_model.predict(Test_Inputs_5)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_5, axis=-1)
    acc_p5_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   


    Perfomance = Perfomance.append({'Feature_Extract': "VGG16",'Size':'20&s2_30','Time':run_time,
                                    'Partcp_Acc_p15':acc_p15_s,'Command_Acc_p15':acc_p15_c,'Partcp_Acc_p1':acc_p1_s,
                                    'Command_Acc_p1':acc_p1_c,'Partcp_Acc_p2':acc_p2_s,'Command_Acc_p2':acc_p2_c,
                                    'Partcp_Acc_p3':acc_p3_s,'Command_Acc_p3':acc_p3_c,'Partcp_Acc_p4':acc_p4_s,
                                    'Command_Acc_p4':acc_p4_c,'Partcp_Acc_p5':acc_p5_s,'Command_Acc_p5':acc_p5_c}, ignore_index=True)


    if high_acc < acc_p15_c + acc_p15_s:
        vgg_model.save('Initial_group_model_VGG_0907.h5')
        high_acc = acc_p15_c + acc_p15_s
        best_index = run
        pd.DataFrame.from_dict(history1.history).to_csv('mix_20&s2_30_history1_Initial_group_model_VGG_0907.csv',index=False)
        pd.DataFrame.from_dict(history2.history).to_csv('mix_20&s2_30_history2_Initial_group_model_VGG_0907.csv',index=False)
        
    del vgg_model
    run = run + 1

Epoch 1/10
18/18 [==============================] - 4s 223ms/step - loss: 3.6362 - participant_output_loss: 1.4467 - command_output_loss: 2.1869 - command_output_1_loss: 4.1286e-04 - participant_output_1_loss: 0.0023 - participant_output_accuracy: 0.3900 - command_output_accuracy: 0.3382 - command_output_1_accuracy: 0.1636 - participant_output_1_accuracy: 0.2518 - val_loss: 3.5653 - val_participant_output_loss: 1.3572 - val_command_output_loss: 2.2075 - val_command_output_1_loss: 1.7395e-04 - val_participant_output_1_loss: 4.7964e-04 - val_participant_output_accuracy: 0.5451 - val_command_output_accuracy: 0.2781 - val_command_output_1_accuracy: 0.5893 - val_participant_output_1_accuracy: 0.0737
Epoch 2/10
18/18 [==============================] - 2s 132ms/step - loss: 3.1750 - participant_output_loss: 1.1802 - command_output_loss: 1.9944 - command_output_1_loss: 9.8970e-05 - participant_output_1_loss: 2.8834e-04 - participant_output_accuracy: 0.5445 - command_output_accuracy: 0.5100 - c

Epoch 3/10
18/18 [==============================] - 2s 133ms/step - loss: 0.8789 - participant_output_loss: 0.2019 - command_output_loss: 0.6769 - command_output_1_loss: 9.5576e-05 - participant_output_1_loss: 3.4516e-05 - participant_output_accuracy: 0.9873 - command_output_accuracy: 0.9918 - command_output_1_accuracy: 0.0655 - participant_output_1_accuracy: 0.1127 - val_loss: 2.5547 - val_participant_output_loss: 0.7554 - val_command_output_loss: 1.7991 - val_command_output_1_loss: 7.8851e-05 - val_participant_output_1_loss: 4.0180e-05 - val_participant_output_accuracy: 0.8140 - val_command_output_accuracy: 0.5285 - val_command_output_1_accuracy: 0.1694 - val_participant_output_1_accuracy: 0.1105
Epoch 4/10
18/18 [==============================] - 3s 170ms/step - loss: 0.7680 - participant_output_loss: 0.1686 - command_output_loss: 0.5993 - command_output_1_loss: 8.2134e-05 - participant_output_1_loss: 3.1428e-05 - participant_output_accuracy: 0.9927 - command_output_accuracy: 0.9891

Epoch 5/10
18/18 [==============================] - 3s 170ms/step - loss: 2.5899 - participant_output_loss: 0.7281 - command_output_loss: 1.8616 - command_output_1_loss: 6.0644e-05 - participant_output_1_loss: 1.2885e-04 - participant_output_accuracy: 0.8591 - command_output_accuracy: 0.7827 - command_output_1_accuracy: 0.0482 - participant_output_1_accuracy: 0.2145 - val_loss: 3.0941 - val_participant_output_loss: 1.0139 - val_command_output_loss: 2.0800 - val_command_output_1_loss: 3.2853e-05 - val_participant_output_1_loss: 1.4336e-04 - val_participant_output_accuracy: 0.7366 - val_command_output_accuracy: 0.4751 - val_command_output_1_accuracy: 0.1639 - val_participant_output_1_accuracy: 0.3499
Epoch 6/10
18/18 [==============================] - 3s 150ms/step - loss: 2.4161 - participant_output_loss: 0.6509 - command_output_loss: 1.7650 - command_output_1_loss: 5.7364e-05 - participant_output_1_loss: 9.8173e-05 - participant_output_accuracy: 0.8818 - command_output_accuracy: 0.8727

Epoch 7/10
18/18 [==============================] - 3s 147ms/step - loss: 0.9551 - participant_output_loss: 0.1616 - command_output_loss: 0.7933 - command_output_1_loss: 4.7602e-05 - participant_output_1_loss: 2.2276e-05 - participant_output_accuracy: 0.9955 - command_output_accuracy: 0.9873 - command_output_1_accuracy: 0.0645 - participant_output_1_accuracy: 0.1209 - val_loss: 2.4695 - val_participant_output_loss: 0.7257 - val_command_output_loss: 1.7437 - val_command_output_1_loss: 4.6660e-05 - val_participant_output_1_loss: 2.7576e-05 - val_participant_output_accuracy: 0.8250 - val_command_output_accuracy: 0.5322 - val_command_output_1_accuracy: 0.3186 - val_participant_output_1_accuracy: 0.2099
Epoch 8/10
18/18 [==============================] - 3s 150ms/step - loss: 0.8781 - participant_output_loss: 0.1443 - command_output_loss: 0.7337 - command_output_1_loss: 5.3455e-05 - participant_output_1_loss: 2.0427e-05 - participant_output_accuracy: 0.9955 - command_output_accuracy: 0.9918

Epoch 9/10
18/18 [==============================] - 3s 165ms/step - loss: 1.7183 - participant_output_loss: 0.3114 - command_output_loss: 1.4067 - command_output_1_loss: 1.2173e-04 - participant_output_1_loss: 1.0632e-04 - participant_output_accuracy: 0.9618 - command_output_accuracy: 0.9364 - command_output_1_accuracy: 0.0055 - participant_output_1_accuracy: 0.1409 - val_loss: 2.7349 - val_participant_output_loss: 0.7876 - val_command_output_loss: 1.9471 - val_command_output_1_loss: 5.1676e-05 - val_participant_output_1_loss: 1.1770e-04 - val_participant_output_accuracy: 0.8066 - val_command_output_accuracy: 0.5193 - val_command_output_1_accuracy: 0.0939 - val_participant_output_1_accuracy: 0.1694
Epoch 10/10
18/18 [==============================] - 3s 155ms/step - loss: 1.5450 - participant_output_loss: 0.2477 - command_output_loss: 1.2971 - command_output_1_loss: 8.9929e-05 - participant_output_1_loss: 9.2660e-05 - participant_output_accuracy: 0.9791 - command_output_accuracy: 0.965

...

Epoch 5/10
18/18 [==============================] - 3s 161ms/step - loss: 2.3493 - participant_output_loss: 0.7101 - command_output_loss: 1.6388 - command_output_1_loss: 9.6600e-05 - participant_output_1_loss: 2.5220e-04 - participant_output_accuracy: 0.8318 - command_output_accuracy: 0.8400 - command_output_1_accuracy: 0.0591 - participant_output_1_accuracy: 0.2136 - val_loss: 2.9878 - val_participant_output_loss: 0.9681 - val_command_output_loss: 2.0193 - val_command_output_1_loss: 8.0596e-05 - val_participant_output_1_loss: 2.3939e-04 - val_participant_output_accuracy: 0.7495 - val_command_output_accuracy: 0.4825 - val_command_output_1_accuracy: 0.0884 - val_participant_output_1_accuracy: 0.1897
Epoch 6/10
18/18 [==============================] - 3s 157ms/step - loss: 2.0960 - participant_output_loss: 0.5795 - command_output_loss: 1.5162 - command_output_1_loss: 1.0980e-04 - participant_output_1_loss: 2.0111e-04 - participant_output_accuracy: 0.8864 - command_output_accuracy: 0.8645

Epoch 7/10
18/18 [==============================] - 2s 131ms/step - loss: 0.5894 - participant_output_loss: 0.1290 - command_output_loss: 0.4603 - command_output_1_loss: 7.7110e-05 - participant_output_1_loss: 4.2796e-05 - participant_output_accuracy: 0.9945 - command_output_accuracy: 0.9945 - command_output_1_accuracy: 0.3255 - participant_output_1_accuracy: 0.2873 - val_loss: 2.4797 - val_participant_output_loss: 0.6919 - val_command_output_loss: 1.7877 - val_command_output_1_loss: 4.8121e-05 - val_participant_output_1_loss: 4.9185e-05 - val_participant_output_accuracy: 0.8195 - val_command_output_accuracy: 0.5341 - val_command_output_1_accuracy: 0.0829 - val_participant_output_1_accuracy: 0.3407
Epoch 8/10
18/18 [==============================] - 2s 125ms/step - loss: 0.5225 - participant_output_loss: 0.1134 - command_output_loss: 0.4090 - command_output_1_loss: 5.8803e-05 - participant_output_1_loss: 3.9154e-05 - participant_output_accuracy: 0.9964 - command_output_accuracy: 0.9964

Epoch 9/10
18/18 [==============================] - 2s 135ms/step - loss: 1.9034 - participant_output_loss: 0.4499 - command_output_loss: 1.4534 - command_output_1_loss: 6.1220e-05 - participant_output_1_loss: 9.4899e-05 - participant_output_accuracy: 0.9336 - command_output_accuracy: 0.9436 - command_output_1_accuracy: 0.0164 - participant_output_1_accuracy: 0.1918 - val_loss: 2.8660 - val_participant_output_loss: 0.8897 - val_command_output_loss: 1.9761 - val_command_output_1_loss: 6.3160e-05 - val_participant_output_1_loss: 1.0003e-04 - val_participant_output_accuracy: 0.7772 - val_command_output_accuracy: 0.5064 - val_command_output_1_accuracy: 0.1529 - val_participant_output_1_accuracy: 0.1455
Epoch 10/10
18/18 [==============================] - 2s 130ms/step - loss: 1.7402 - participant_output_loss: 0.3849 - command_output_loss: 1.3551 - command_output_1_loss: 5.8390e-05 - participant_output_1_loss: 9.1430e-05 - participant_output_accuracy: 0.9573 - command_output_accuracy: 0.949

Epoch 1/10
18/18 [==============================] - 3s 174ms/step - loss: 3.6888 - participant_output_loss: 1.4318 - command_output_loss: 2.2539 - command_output_1_loss: 9.2867e-04 - participant_output_1_loss: 0.0022 - participant_output_accuracy: 0.4127 - command_output_accuracy: 0.2445 - command_output_1_accuracy: 0.0218 - participant_output_1_accuracy: 0.2364 - val_loss: 3.5862 - val_participant_output_loss: 1.3714 - val_command_output_loss: 2.2139 - val_command_output_1_loss: 1.5581e-04 - val_participant_output_1_loss: 7.4390e-04 - val_participant_output_accuracy: 0.4604 - val_command_output_accuracy: 0.2597 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.0866
Epoch 2/10
18/18 [==============================] - 2s 131ms/step - loss: 3.2115 - participant_output_loss: 1.1349 - command_output_loss: 2.0760 - command_output_1_loss: 9.4295e-05 - participant_output_1_loss: 4.8938e-04 - participant_output_accuracy: 0.5973 - command_output_accuracy: 0.4636

Epoch 3/10
18/18 [==============================] - 3s 161ms/step - loss: 1.0643 - participant_output_loss: 0.2218 - command_output_loss: 0.8424 - command_output_1_loss: 1.0384e-04 - participant_output_1_loss: 5.7963e-05 - participant_output_accuracy: 0.9864 - command_output_accuracy: 0.9818 - command_output_1_accuracy: 0.1118 - participant_output_1_accuracy: 0.1555 - val_loss: 2.5384 - val_participant_output_loss: 0.7536 - val_command_output_loss: 1.7846 - val_command_output_1_loss: 7.2506e-05 - val_participant_output_1_loss: 7.5306e-05 - val_participant_output_accuracy: 0.8048 - val_command_output_accuracy: 0.5378 - val_command_output_1_accuracy: 0.0147 - val_participant_output_1_accuracy: 0.1473
Epoch 4/10
18/18 [==============================] - 3s 155ms/step - loss: 0.9551 - participant_output_loss: 0.1953 - command_output_loss: 0.7596 - command_output_1_loss: 8.7468e-05 - participant_output_1_loss: 5.7050e-05 - participant_output_accuracy: 0.9882 - command_output_accuracy: 0.9873

In [18]:
vgg_model = tf.keras.models.load_model('Initial_group_model_VGG_0907.h5')
predictions = vgg_model.predict(Test_Inputs)[0]
predicted_classes = np.argmax(predictions, axis=1)+1
acc_s = round(sum(x == y for x, y in zip(test_unit_participant_class, predicted_classes)) / len(test_unit_participant_class),4)

predictions = vgg_model.predict(Test_Inputs)[1]
predicted_classes = np.argmax(predictions, axis=1)
acc_c = round(sum(x == y for x, y in zip(test_unit_command_class, predicted_classes)) / len(test_unit_command_class),4)
overall_acc = acc_c + acc_s
print(acc_s,acc_c)

0.8803 0.9263


In [19]:
Perfomance

,Feature_Extract,Time,Partcp_Acc_p15,Command_Acc_p15,Partcp_Acc_p1,Command_Acc_p1,Partcp_Acc_p2,Command_Acc_p2,Partcp_Acc_p3,Command_Acc_p3,Partcp_Acc_p4,Command_Acc_p4,Partcp_Acc_p5,Command_Acc_p5,Size
0,VGG16,63.237659,0.8895,0.9134,0.9189,0.9279,0.9722,0.7685,0.9217,0.9130,0.7615,0.9725,0.87,0.99,20&s2_30
1,VGG16,62.212071,0.8803,0.9116,0.8649,0.9189,0.9722,0.7685,0.9304,0.9304,0.7339,0.9633,0.90,0.98,20&s2_30
2,VGG16,61.656819,0.8840,0.9061,0.9009,0.9279,0.9444,0.7593,0.9304,0.9217,0.7798,0.9541,0.86,0.97,20&s2_30
3,VGG16,61.511568,0.8803,0.9263,0.8649,0.9369,0.9537,0.7963,0.8870,0.9478,0.7706,0.9725,0.93,0.98,20&s2_30
4,VGG16,60.123125,0.8785,0.9171,0.7928,0.9009,0.9537,0.7870,0.9043,0.9478,0.8440,0.9817,0.90,0.97,20&s2_30
5,VGG16,59.735848,0.8840,0.9190,0.9099,0.9279,0.9444,0.7870,0.9217,0.9304,0.7431,0.9725,0.90,0.98,20&s2_30
6,VGG16,57.168746,0.8803,0.9079,0.8468,0.9189,0.9444,0.7685,0.9304,0.9130,0.7798,0.9725,0.90,0.97,20&s2_30
7,VGG16,57.110326,0.8748,0.9042,0.8288,0.8919,0.9074,0.7593,0.8957,0.9391,0.8440,0.9725,0.90,0.96,20&s2_30
8,VGG16,57.662082,0.8766,0.9098,0.9009,0.9279,0.9537,0.7593,0.8957,0.9478,0.7706,0.9450,0.86,0.97,20&s2_30
9,VGG16,58.186677,0.8895,0.9061,0.8559,0.9009,0.9537,0.7685,0.9478,0.9130,0.7890,0.9725,0.90,0.98,20&s2_30


In [20]:
Perfomance.to_csv('Performance_0907_Feature_extract.csv',index=False)

In [21]:
best_index

3